In [1]:
import wandb
import torch
# from utils2 import eval_zs_gzsl
from helper_func import eval_zs_gzsl
from dataset_test import AWA2DataLoader
import argparse
import json
import pickle
import os
import pickle
from torchvision import transforms
from torch.utils.data import Dataset, Subset, DataLoader
from PIL import Image
import sys

import torch.optim as optim
import torch.nn as nn
import numpy as np
from model_test import BDA
# from dataset import AWA2DataLoader
from  helper_func import eval_zs_gzsl
import json
import argparse

In [2]:
dataloader = AWA2DataLoader('.', "cuda:0")

.
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
AWA2
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Balance dataloader
_____
./../revice_Transzero/data/AWA2/feature_map_ResNet_101_AWA2.hdf5
Expert Attr
threshold at zero attribute with negative value


In [3]:
sweep_configuration = {
    'method': 'grid',
    'name': ' test_awa2_gzsl',
    'metric': {'goal': 'maximize', 'name': 'best_H'},
    'parameters': 
    {  'batch_size':    {'value': 50},
        'dataset': { 'value': 'AWA2'},
        'device': {'value': 'cuda:0'},
        'dim_f': { 'value': 2048},
        'dim_v':{ 'value': 300},
        'epochs': {'value': 130},
        'img_size': {'value': 448},
        'lambda_': {'values': [2.5]},
        'lambda_reg': {'values': [0.0001]},
        'lambda_CEVi': {'values': [0.0025]},
        'normalize_V': {'value': "False"},
        'num_attribute': { 'value': 85},
        'num_class': {'value': 50},
        'random_seed': {'value': 17},
        'tf_SAtt':  {'value':  "true"},
        'tf_aux_embed': {'value': "True"},
        'tf_common_dim': {'values': [500]},  
        'tf_dc_layer': {'value': 1},
        'tf_dim_feedforward': {'values': [2048]},
        'tf_dropout': {'values':[0.45]},
        'tf_dropout1': {'values': [0.45]},        

        'tf_ec_layer': {'value':1},
        'tf_heads': {'value': 1},
        'alpha': {'value': 1},
        'beta' : {'value': 1},
        'fsqrt': {'values': [127]},
        'sqrt': {'values': [ 153]},        
        'weight' : {'values': [0.01]} ,
        'weight_self' : {'values': [1]},
        'tgt_weight' : {'values': [1]}                       
        }
                                                }
sweep_id = wandb.sweep(sweep=sweep_configuration, project='Ko_bda')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: nca3g67a
Sweep URL: https://wandb.ai/junseoklee_333/Ko_bda/sweeps/nca3g67a


In [4]:
def main():
    run = wandb.init()
    config = wandb.config
    # set random seed
    seed = config.random_seed
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

    # TransZero model
    model = BDA(config, dataloader.att, dataloader.w2v_att,
                    dataloader.seenclasses, dataloader.unseenclasses).to(config.device)
    
    # main loop
    niters = dataloader.ntrain * config.epochs//config.batch_size
    report_interval = niters//config.epochs
    best_performance = [0, 0, 0, 0]
    best_performance_zsl = 0
    model_dict = model.state_dict()
    # saved_dict = torch.load(config.saved_model)
    check_point = torch.load("./checkpoint/AWA2/AWA2_gzsl.pth")
    saved_dict = check_point["model_state_dict"]
    saved_dict = {k: v for k, v in saved_dict.items() if k in model_dict}
    model_dict.update(saved_dict)
    model.load_state_dict(model_dict)
    model.to(config.device)
    model.eval()

    # report result

    print('-'*30)
    acc_seen, acc_novel, H, acc_zs = eval_zs_gzsl(
        dataloader, model, config.device, bias_seen=0, bias_unseen=0)

    if H > best_performance[2]:
        best_performance = [acc_novel, acc_seen, H, acc_zs]

    print('acc_unseen=%.3f, acc_seen=%.3f, H=%.3f | '
        'acc_zs=%.3f' % (  best_performance[0], best_performance[1],
            best_performance[2], best_performance_zsl))

    wandb.log({
        'best_acc_unseen': best_performance[0],
        'best_acc_seen': best_performance[1],
        'best_H': best_performance[2]
        })

In [5]:
wandb.agent(sweep_id, function=main, count=2 )

wandb: Agent Starting Run: g35mkxpo with config:
wandb: 	alpha: 1
wandb: 	batch_size: 50
wandb: 	beta: 1
wandb: 	dataset: AWA2
wandb: 	device: cuda:0
wandb: 	dim_f: 2048
wandb: 	dim_v: 300
wandb: 	dropout_self: 1
wandb: 	epochs: 130
wandb: 	fsqrt: 127
wandb: 	img_size: 448
wandb: 	lambda_: 2.5
wandb: 	lambda_CEVi: 0.0025
wandb: 	lambda_reg: 0.0001
wandb: 	normalize_V: False
wandb: 	num_attribute: 85
wandb: 	num_class: 50
wandb: 	random_seed: 17
wandb: 	sqrt: 153
wandb: 	tf_SAtt: true
wandb: 	tf_aux_embed: True
wandb: 	tf_common_dim: 500
wandb: 	tf_dc_layer: 1
wandb: 	tf_dim_feedforward: 2048
wandb: 	tf_dropout: 0.45
wandb: 	tf_dropout1: 0.45
wandb: 	tf_ec_layer: 1
wandb: 	tf_heads: 1
wandb: 	tgt_weight: 1
wandb: 	weight: 0.01
wandb: 	weight_self: 1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: junseoklee_333. Use `wandb login --relogin` to force relogin


------------------------------
acc_unseen=0.629, acc_seen=0.797, H=0.703 | acc_zs=0.000


best_H,▁
best_acc_seen,▁
best_acc_unseen,▁
best_H,0.70337
best_acc_seen,0.79742
best_acc_unseen,0.62916


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
